In [1]:
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

data = '{ "resource_ids": [ 39435, 39426, 39425, 38657 ], "wikification_type": "SIMPLE"}'

response = requests.post('http://wp3dev.x5gon.org/distance/wikifier/fetch', headers=headers, data=data)


In [2]:
import json
response_c = response.content
dict_response = json.loads(response_c.decode('utf-8'))

In [25]:
import itertools
import math
import trueskill as t
import numpy as np
import json
import operator

def return_topic_dificulty(material_id, n_topics = 10):
    headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    }

    data ='{ "resource_ids": ['+str(material_id)+'], "wikification_type": "SIMPLE"}'

    response = requests.post('http://wp3dev.x5gon.org/distance/wikifier/fetch', headers=headers, data=data)
    concepts = json.loads(response.content.decode())['output'][0]['value']['concepts']
    concepts_dificulties = {i['title']:i['cosine'] for i in concepts}
    
    
    return {k: v for k, v in sorted(concepts_dificulties.items(), key=lambda item: item[1])[::-1][:n_topics]}

class TrueBuddy_learner:
    def __init__(self, starting_skill = 0, starting_var = (25/3), BETA = 25/6, inf = 1/1000):
        #starting skill is defenitly 0, starting var has to be determined
        self.st_skill = starting_skill
        self.st_var = starting_var
        self.BETA = BETA
        self.learners = {}
        self.inf = inf
        
    
    
    def update_skills(self, material_id, feedback):
        skills = return_topic_dificulty(material_id)
        for skill in skills.keys():
            if skill in self.learners:
                pass
            else:
                self.learners[skill] = t.Rating(self.st_skill,self.st_var)
        active_list = [self.learners[skill] for skill in skills.keys()]
        
        resource_list = [t.Rating(skills[skill],self.inf) for skill in skills.keys()]
        #print(resource_list)
        
        if feedback == 1:
            active_list, resource_list = t.rate([active_list,resource_list],ranks=[0,1])
        elif feedback == 0:
            active_list, resource_list = t.rate([active_list,resource_list],ranks=[1,0])
        else:
            raise ValueError('feedback can only be 1 or 0')
        
        #print(active_list)
        
        skill_list = list(skills.keys())
        for i in range(len(skill_list)):
            self.learners[skill_list[i]] = active_list[i]
        
        return None

    def win_probability(self,team1, team2):
        delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
        sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
        size = len(team1) + len(team2)
        denom = math.sqrt(size * (self.BETA * self.BETA) + sum_sigma)
        ts = t.global_env()
        return ts.cdf(delta_mu / denom)

    
    
    
    
    
    def predict_proba(self,material_id):
        skills = return_topic_dificulty(material_id)
        for skill in skills.keys():
            if skill in self.learners:
                pass
            else:
                self.learners[skill] = t.Rating(self.st_skill,self.st_var)
        active_list = [self.learners[skill] for skill in skills.keys()]
        
        resource_list = [t.Rating(skills[skill],self.inf) for skill in skills.keys()]
        #print(resource_list)
        
        return(self.win_probability(active_list,resource_list))
        
    
    def predict(self,material_id):
        win_proba = self.predict_proba(material_id)
        
        return(1 if win_proba >= 0.5 else 0)
        
        

In [24]:
td = return_topic_dificulty(39435)
{k: v for k, v in sorted(td.items(), key=lambda item: item[1])[::-1][:10]}

{'Human': 0.3224942985,
 'Shapeshifting': 0.3082210289,
 'Justice': 0.2955489987,
 'Education': 0.2903795047,
 'Lawsuit': 0.2886705463,
 'Psychology': 0.2884764426,
 'Free will': 0.2851233192,
 'Thought': 0.2825369196,
 'Anatomical terms of location': 0.2810134195,
 'Computer': 0.2766288251}

In [4]:
l1 = TrueBuddy_learner()

In [5]:
for i in range(1):
    l1.update_skills(39435,1)
l1.predict_proba(39435)

0.675369051331635

In [10]:
import TrueBuddy as TB

In [11]:
l2 = TB.TrueBuddy_learner()

In [26]:
for i in range(10):
    l2.update_skills(39435,1)
    print(l2.predict_proba(39425))


0.7439710544921114
0.7506698953680635
0.7566612813407494
0.7620636319642924
0.7669693407756536
0.7714516791165023
0.7755695921929247
0.7793711058966719
0.782895795059081
0.786176603510017


In [30]:
for i in range(10):
    l2.update_skills(39425,0)
    print(l2.predict_proba(39425))


0.6406735366563173
0.3569778347099068
0.21783087318456615
0.14650020734100486
0.10597721094691515
0.08084815314007884
0.06416491302529449
0.052486042183002436
0.043961396061514973
0.037526561843419744


In [31]:
l2.learners

{"Prisoner's dilemma": trueskill.Rating(mu=1.176, sigma=8.313),
 'Vampire bat': trueskill.Rating(mu=1.176, sigma=8.313),
 'Psychology': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Vampire': trueskill.Rating(mu=1.176, sigma=8.313),
 'Evolution': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Culture of honor (Southern United States)': trueskill.Rating(mu=1.176, sigma=8.313),
 'Human': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Operant conditioning': trueskill.Rating(mu=1.176, sigma=8.313),
 'Primate': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Emotion': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Bat': trueskill.Rating(mu=1.176, sigma=8.313),
 'Reciprocal altruism': trueskill.Rating(mu=1.176, sigma=8.313),
 'Ultimatum game': trueskill.Rating(mu=1.176, sigma=8.313),
 'Sociology': trueskill.Rating(mu=1.176, sigma=8.313),
 'Computer': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Gene': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Education': trueskill.Rating(mu=-0.121, sigma=8.276),
 'Knowled

In [15]:
l3 = TB.TrueBuddy_learner()

In [16]:
l3.learners

{}